using the napari tracks layers : https://napari.org/stable/howtos/layers/tracks.html 


In [1]:
import napari 

In [2]:
v = napari.Viewer()

In [3]:
import numpy as np 

In [4]:
v.add_tracks?

Signature:
v.add_tracks(
    data,
    *,
    features=None,
    properties=None,
    graph=None,
    tail_width=2,
    tail_length=30,
    head_length=0,
    name=None,
    metadata=None,
    scale=None,
    translate=None,
    rotate=None,
    shear=None,
    affine=None,
    opacity=1,
    blending='additive',
    visible=True,
    colormap='turbo',
    color_by='track_id',
    colormaps_dict=None,
    cache=True,
    experimental_clipping_planes=None,
) -> napari.layers.tracks.tracks.Tracks
Docstring:
Add a Tracks layer to the layer list. 

Parameters
----------
data : array (N, D+1)
    Coordinates for N points in D+1 dimensions. ID,T,(Z),Y,X. The first
    axis is the integer ID of the track. D is either 3 or 4 for planar
    or volumetric timeseries respectively.
features : Dataframe-like
    Features table where each row corresponds to a point and each column
    is a feature.
properties : dict {str: array (N,)}, DataFrame
    Properties for each point. Each property should be 

## biased random walk 

The following function takes a fixed number of particles and projects forward in time using a biased random walk. At each timestep, the trajectory is a weighted average of the previous timestep's trajectory with a new trajectory (of random orientation)

In [5]:
def biased_random_walk_tracks(n_particles, n_timesteps, image_size_xyz, bias_old = 0.8):    
    # random walk with biases towards trajectory from prior timestep. particles are 
    # individual and persistent 
    track_data = []
    ids = np.arange(n_particles, dtype='int').transpose()
    
    # initialize start positions     
    xyz = np.random.random((n_particles, 3))
    t = np.zeros((n_particles,))

    # track array columns: ID,T,(Z),Y,X
    current_track = np.column_stack([ids, t, xyz[:, 2], xyz[:, 1], xyz[:, 0]])
    track_data.append(current_track)      
    
    for it in range(n_timesteps):
        d_xyz = (np.random.random((n_particles, 3)) - 0.5) * 0.1

        if it > 0:
            d_xyz = d_xyz0 * bias_old + d_xyz * (1 - bias_old)
        
        xyz = xyz + d_xyz 
        t = t + 1 # arbitrary time
        current_track = np.column_stack([ids, t, xyz[:, 2], xyz[:, 1], xyz[:, 0]])
        track_data.append(current_track)
        
        d_xyz0 = d_xyz

    track_data = np.concatenate(track_data)

    # scale final position ranges
    for idim in range(3): 
        mx_dim = track_data[:, idim+2].max()
        mn_dim = track_data[:, idim+2].min()
        scaled = (track_data[:, idim+2] - mn_dim)/ (mx_dim - mn_dim)
        track_data[:, idim+2] = scaled * image_size_xyz[idim]

    track_data = track_data[track_data[:, 0].argsort()]#.astype('int')
    
    return track_data

In [6]:
n_particles = 50
timesteps = 200
tracks = biased_random_walk_tracks(n_particles, timesteps, (400, 400, 400))
tracks.shape

(10050, 5)

In [7]:
v.add_tracks(tracks)

<Tracks layer 'tracks' at 0x2be5d7f70>

notes on viewing in napari:

in 2D view, all tracks are projected to the 2d plane (https://github.com/napari/napari/issues/3861)


## biased random walk with particle generation 

Same as before, but now there's a nonzero chance of each particle spawning a new particle. The new particle starts with the same position and old trajectory (which will be weighted with a new, different trajectory)

Still just using `np.random.random`.... a new particle is generated if `np.random.random() < gen_particle_coeff`

In [8]:
def biased_random_walk_tracks_with_generation(n_starting, 
                                              n_max_particles, 
                                              n_timesteps, 
                                              image_size_xyz, 
                                              bias_old = 0.8,
                                              gen_particle_coeff=0.05):    
    # random walk with biases towards trajectory from prior timestep. particles are 
    # individual and persistent 
    track_data = []
    
    # initialize start positions     
    xyz = np.random.random((n_starting, 3))
    t = np.zeros((n_starting,))
    ids = np.arange(n_starting, dtype='int').transpose()

    # track array columns: ID,T,(Z),Y,X
    current_track = np.column_stack([ids, t, xyz[:, 2], xyz[:, 1], xyz[:, 0]])
    track_data.append(current_track)

    n_particles = n_starting
    graph = {}
    for it in range(n_timesteps):

        # particle generation
        generate_draw = np.random.random((n_particles,)) < gen_particle_coeff
        n_new_particles = np.sum(generate_draw)
        
        if n_new_particles  > 0 and n_particles < n_max_particles:
            # graph : dict {int: list}
            # Graph representing associations between tracks. Dictionary defines the
            # mapping between a track ID and the parents of the track. This can be
            # one (the track has one parent, and the parent has >=1 child) in the
            # case of track splitting, or more than one (the track has multiple
            # parents, but only one child) in the case of track merging.
            # See examples/tracks_3d_with_graph.py
            parent_ids = ids[generate_draw]
            new_ids = np.arange(len(ids), len(ids)+n_new_particles, dtype='int').transpose()
            for id, parent_id in zip(new_ids, parent_ids):
                # child points to parent
                graph[id] = parent_id
                            
            ids = np.concatenate([ids, new_ids])
            xyz = np.concatenate([xyz, xyz[generate_draw]])            
            t = np.concatenate([t, t[generate_draw]])                        
            n_particles = len(ids)
            if it > 0: 
                d_xyz0 = np.concatenate([d_xyz0, d_xyz0[generate_draw]])
            
        
        d_xyz = (np.random.random((n_particles, 3)) - 0.5) * 0.1

        if it > 0:
            d_xyz = d_xyz0 * bias_old + d_xyz * (1 - bias_old)
        
        xyz = xyz + d_xyz 
        t = t + 1 # arbitrary time
        current_track = np.column_stack([ids, t, xyz[:, 2], xyz[:, 1], xyz[:, 0]])
        track_data.append(current_track)
        
        d_xyz0 = d_xyz

    track_data = np.concatenate(track_data)

    # scale final position ranges
    for idim in range(3): 
        mx_dim = track_data[:, idim+2].max()
        mn_dim = track_data[:, idim+2].min()
        scaled = (track_data[:, idim+2] - mn_dim)/ (mx_dim - mn_dim)
        track_data[:, idim+2] = scaled * image_size_xyz[idim]

    track_data = track_data[track_data[:, 0].argsort()]#.astype('int')
    
    return track_data, graph

In [9]:
n_particles = 2
n_max_particles = 100
timesteps = 200
tracks, graph = biased_random_walk_tracks_with_generation(n_particles, 
                                                   n_max_particles,
                                                   timesteps, 
                                                   (400, 400, 400), 
                                                    gen_particle_coeff=0.03,
                                                    )
tracks.shape

(9819, 5)

In [10]:
v.layers.clear()
v.add_tracks(tracks, graph=graph)

<Tracks layer 'tracks' at 0x2be5a71f0>

## biased random walk with annihilation 

* particles falling within specified distance collapse into 1 new survivor
* fixed, sticky boundary: particles projected past box edges are moved back, just a simple way to increase interactions (reflective boundary would be nicer)

In [11]:
import numpy as np 

def biased_random_walk_tracks_with_annihilation(n_starting,                                               
                                              n_timesteps, 
                                              image_size_xyz, 
                                              bias_old = 0.8,
                                              annihilation_dist=0.05):    
    # random walk with biases towards trajectory from prior timestep. particles are 
    # individual and persistent 
    track_data = []
    
    # initialize start positions     
    xyz = np.random.random((n_starting, 3))
    t = np.zeros((n_starting,))
    ids = np.arange(n_starting, dtype='int').transpose()
    surviving = ids >= 0
    # track array columns: ID,T,(Z),Y,X
    current_track = np.column_stack([ids, t, xyz[:, 2], xyz[:, 1], xyz[:, 0]])
    track_data.append(current_track)

    n_particles = n_starting
    graph = {}
    for it in range(n_timesteps):
        # print(f"{len(ids[surviving])} remain")
        # find points within annihilation_dist of each other. this is hella slow and ugly but whatever        
        ids_to_rm = []
        survivors = []
        survivor_parents = {}
        for id in ids[surviving]:            
            if id not in ids_to_rm: 
                dist = np.zeros(ids.shape)
                for idim in range(3):
                    dist += (xyz[:, idim] - xyz[id, idim])**2
                dist = np.sqrt(dist)                
                close_enough = (dist < annihilation_dist) & (surviving)                
                if np.sum(close_enough) > 1: 
                    close_ids = ids[close_enough]
                    survivors.append(id)
                    survivor_parents[id] = [idc for idc in close_ids]
                    ids_to_rm += [idc for idc in close_ids]

        # add particles for the survivor
        surviving[ids_to_rm] = False 
        new_particles = len(survivors)
        if new_particles > 0:
            new_ids = []            
        
            new_id = len(ids) - 1
            for survivor in survivors:
                new_id += 1
                new_ids.append(new_id)
                
                graph[new_id] = survivor_parents[survivor]

            ids = np.concatenate([ids, new_ids])
            xyz = np.concatenate([xyz, xyz[survivors,:]])            
            t = np.concatenate([t, t[survivors]])                        
            surviving = np.concatenate([surviving, [True,] * len(new_ids)])  
            n_particles = len(ids)
            if it > 0: 
                d_xyz0 = np.concatenate([d_xyz0, d_xyz0[survivors,:]])
 
        d_xyz = (np.random.random((n_particles, 3)) - 0.5) * 0.1

        if it > 0:
            d_xyz = d_xyz0 * bias_old + d_xyz * (1 - bias_old)
        
        xyz = xyz + d_xyz

        # pin to boundary (avoid periodic tail effect)
        exited_right = xyz > 1.0 
        exited_left = xyz < 0.0
        xyz[exited_right] = 1.0 
        xyz[exited_left] = 0.0
        
        t = t + 1 # arbitrary time
        current_track = np.column_stack([ids[surviving], t[surviving], xyz[surviving, 2], xyz[surviving, 1], xyz[surviving, 0]])
        track_data.append(current_track)
        
        d_xyz0 = d_xyz

    track_data = np.concatenate(track_data)

    # scale final position ranges
    for idim in range(3): 
        mx_dim = track_data[:, idim+2].max()
        mn_dim = track_data[:, idim+2].min()
        scaled = (track_data[:, idim+2] - mn_dim)/ (mx_dim - mn_dim)
        track_data[:, idim+2] = scaled * image_size_xyz[idim]

    track_data = track_data[track_data[:, 0].argsort()]#.astype('int')
    
    return track_data, graph

In [12]:
n_particles = 1000
timesteps = 200
tracks, graph = biased_random_walk_tracks_with_annihilation(n_particles,                                                    
                                                   timesteps, 
                                                   (400, 400, 400), 
                                                    annihilation_dist = 0.05
                                                    )
tracks.shape

(37478, 5)

In [13]:
v.layers.clear()
v.add_tracks(tracks, graph=graph)

<Tracks layer 'tracks' at 0x2c5b96f50>